In [1]:
import pandas as pd

In [6]:
# https://www.kaggle.com/datasets/cosmos98/twitter-and-reddit-sentimental-analysis-dataset?resource=download
dfT = pd.read_csv('archive/Twitter_Data.csv')
dfR = pd.read_csv('archive/Reddit_Data.csv')

In [19]:
dfT.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [20]:
dfR.head()

,clean_text,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [21]:
dfR.rename(columns={"clean_comment":"clean_text"},inplace=True)
df = pd.concat((dfT,dfR))
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [23]:
df.dropna(axis=0,inplace=True)

In [28]:
df.category.value_counts()

 1.0    88079
 0.0    68253
-1.0    43786
Name: category, dtype: int64

In [34]:
print('df size: ', df.shape[0])
minCatCount = min(df.category.value_counts())
print('Min category count: ', minCatCount)

df size:  200118
Min category count:  43786


In [42]:
balanced_df = pd.concat((df[df.category == -1][:minCatCount],df[df.category == 0][:minCatCount],df[df.category == 1][:minCatCount]))
balanced_df.category.value_counts()

-1.0    43786
 0.0    43786
 1.0    43786
Name: category, dtype: int64

In [51]:
from sklearn.model_selection import train_test_split
comment, category = [],[]
for row in df.iterrows():
    comment.append(row[1]['clean_text'])
    category.append(row[1]['category'])

train_X, test_X, train_y, test_y = train_test_split([i for i in comment], [i for i in category], test_size=0.25, random_state=1)

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [61]:
vectorizer = TfidfVectorizer()
train_X_vectors = vectorizer.fit_transform(train_X)
test_X_vectors = vectorizer.transform(test_X)

In [63]:
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
clf_svm = LinearSVC()
clf_svm.fit(train_X_vectors, train_y)
clf_prediction = clf_svm.predict(test_X_vectors)

print(f'Accuracy: {clf_svm.score(test_X_vectors, test_y)}')
print(f'Accuracy: {f1_score(test_y, clf_prediction, average=None, labels=[-1, 0, 1])}')

Accuracy: 0.9462922246652009
Accuracy: [0.9009009  0.9703079  0.94974083]


In [67]:
clf_svm.predict(vectorizer.transform(['How about this text?']))

array([0.])

In [68]:
clf_svm.predict(vectorizer.transform(['This is an awful text']))

array([-1.])

In [70]:
clf_svm.predict(vectorizer.transform(['This is a wonderful text']))

array([1.])